In [1]:
import json
import spacy
from pathlib import Path

# Lade spaCy Modell (oder ersatzweise nlp = spacy.blank("de"))
nlp = spacy.load("de_core_news_md")

# Eingabe- und Ausgabe-Dateien
input_path = Path("ground_truth.json")
output_path = Path("ground_truth_cleaned.json")

def clean_labels(entry, nlp):
    text = entry["text"]
    labels = entry.get("labels", [])
    doc = nlp(text)
    cleaned_labels = []

    for label in labels:
        span = doc.char_span(label["start"], label["end"], label=label["label"], alignment_mode="expand")
        if span:
            cleaned_labels.append({
                "start": span.start_char,
                "end": span.end_char,
                "label": span.label_
            })
        else:
            print(f"⚠️ Label nicht ausrichtbar in {entry.get('file', '??')}: '{text[label['start']:label['end']]}' @ {label['start']}-{label['end']}")

    return {
        "file": entry.get("file", ""),
        "text": text,
        "labels": cleaned_labels
    }

def process_file(input_path, output_path):
    with input_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data = [clean_labels(entry, nlp) for entry in data]

    with output_path.open("w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Bereinigte Datei gespeichert unter: {output_path.resolve()}")

# Ausführen
process_file(input_path, output_path)


✅ Bereinigte Datei gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/data/original/ground_truth_cleaned.json


In [2]:
import json
import spacy
from pathlib import Path

# Lade spaCy Modell
nlp = spacy.load("de_core_news_md")

# Pfade
input_path = Path("../../data/synthetic/synthetic_mails_option_b.json")
output_path = Path("../../data/synthetic/synthetic_mails_option_b_cleaned.json")


def clean_labels(entry, nlp):
    text = entry["text"]
    labels = entry.get("labels", [])
    doc = nlp(text)
    cleaned_labels = []

    for label in labels:
        start, end, label_name = label["start"], label["end"], label["label"]
        span_exact = doc.char_span(start, end, label=label_name, alignment_mode="strict")
        span_expand = doc.char_span(start, end, label=label_name, alignment_mode="expand")

        if span_exact:
            # Exakt zuweisbar
            cleaned_labels.append({
                "start": span_exact.start_char,
                "end": span_exact.end_char,
                "label": span_exact.label_
            })
        elif span_expand:
            # Zeige was expand erfassen würde
            print(f"🔍 Nicht exakt ausrichtbar ({entry.get('file', '??')}): '{text[start:end]}' @ {start}-{end}")
            print(f"➡️  Würde erweitert auf: '{span_expand.text}' @ {span_expand.start_char}-{span_expand.end_char}")
            print(f"    Tokens: {[t.text for t in span_expand]}")
            print("    ❓ Akzeptieren oder anpassen?\n")

            # Du kannst hier entscheiden, ob du trotzdem übernehmen willst
            # Zum Beispiel temporär automatisch übernehmen:
            cleaned_labels.append({
                "start": span_expand.start_char,
                "end": span_expand.end_char,
                "label": span_expand.label_
            })
        else:
            print(f"⚠️ Gar nicht zuweisbar ({entry.get('file', '??')}): '{text[start:end]}' @ {start}-{end}")

    return {
        "file": entry.get("file", ""),
        "text": text,
        "labels": cleaned_labels
    }

def process_file(input_path, output_path):
    with input_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data = [clean_labels(entry, nlp) for entry in data]

    with output_path.open("w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Bereinigte Datei gespeichert unter: {output_path.resolve()}")

# Start
process_file(input_path, output_path)

🔍 Nicht exakt ausrichtbar (13): '404107498486' @ 53-65
➡️  Würde erweitert auf: '404107498486.' @ 53-66
    Tokens: ['404107498486.']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (16): 'Dipl.-Ing' @ 121-130
➡️  Würde erweitert auf: 'ADipl.-Ingesse' @ 120-134
    Tokens: ['ADipl.-Ingesse']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (16): 'Dr' @ 301-303
➡️  Würde erweitert auf: 'Dr.' @ 301-304
    Tokens: ['Dr.']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (18): 'Aurelia' @ 340-347
➡️  Würde erweitert auf: 'Aurelia_Löffler' @ 340-355
    Tokens: ['Aurelia_Löffler']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (18): 'Löffler' @ 348-355
➡️  Würde erweitert auf: 'Aurelia_Löffler' @ 340-355
    Tokens: ['Aurelia_Löffler']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (21): '87-86' @ 190-195
➡️  Würde erweitert auf: '087-86' @ 189-195
    Tokens: ['087', '-', '86']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausr

In [8]:
import json
import spacy
from pathlib import Path

# Lade SpaCy-Modell
nlp = spacy.load("de_core_news_md")

# Pfade
input_path   = Path("../../data/synthetic/synthetic_mails_option_a.json")
cleaned_path = Path("../../data/synthetic/synthetic_mails_option_a_cleaned.json")
failed_path  = Path("../../data/synthetic/synthetic_mails_option_a_failed.json")

def try_expand_one_char(doc, text, start, end, label_name):
    # ein Zeichen links
    if start > 0:
        span = doc.char_span(start-1, end, label=label_name, alignment_mode="strict")
        if span:
            return span
    # ein Zeichen rechts
    if end < len(text):
        span = doc.char_span(start, end+1, label=label_name, alignment_mode="strict")
        if span:
            return span
    return None

def clean_labels(entry, nlp, failed_records):
    text   = entry.get("text", "")
    labels = entry.get("labels", [])
    doc    = nlp(text)
    cleaned = []

    for label in labels:
        start, end, label_name = label["start"], label["end"], label["label"]
        # 1) strict
        span = doc.char_span(start, end, label=label_name, alignment_mode="strict")
        if span:
            cleaned.append({
                "start": span.start_char,
                "end":   span.end_char,
                "label": span.label_
            })
            continue

        # 2) single-char expand
        span = try_expand_one_char(doc, text, start, end, label_name)
        if span:
            cleaned.append({
                "start": span.start_char,
                "end":   span.end_char,
                "label": span.label_
            })
            continue

        # 3) token-extend fallback: nur vorschlagen, nicht übernehmen
        span_fb = doc.char_span(start, end, label=label_name, alignment_mode="expand")
        if span_fb:
            suggested = {
                "start": span_fb.start_char,
                "end":   span_fb.end_char,
                "text":  span_fb.text
            }
        else:
            suggested = None

        # 4) sammle Failure mit Vorschlag
        failed_records.append({
            "file":         entry.get("file", ""),
            "orig_label": {
                "start": start,
                "end":   end,
                "label": label_name,
                "text":  text[start:end]
            },
            "suggestion": suggested,
            "context": text  # optional: gesamte Mail
        })

    return {
        "file":   entry.get("file", ""),
        "text":   text,
        "labels": cleaned
    }

def process_file(input_path, cleaned_path, failed_path):
    with input_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data   = []
    failed_records = []

    for entry in data:
        cleaned_entry = clean_labels(entry, nlp, failed_records)
        cleaned_data.append(cleaned_entry)

    # speichern
    with cleaned_path.open("w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)
    with failed_path.open("w", encoding="utf-8") as f:
        json.dump(failed_records, f, ensure_ascii=False, indent=2)

    # Reporting
    total_labels   = sum(len(e.get("labels",[])) + 1 for e in data)  # rough total
    total_failures = len(failed_records)
    print(f"\n✅ Bereinigt: {len(cleaned_data)} Einträge")
    print(f"⚠️ Fehlgeschlagen: {total_failures} Label-Zuordnungen\n")

    if total_failures:
        print("Beispiele der fehlgeschlagenen Labels mit Vorschlag:")
        for rec in failed_records[:5]:
            orig = rec["orig_label"]
            print(f" - Datei {rec['file']!r}:")
            print(f"    • Original: '{orig['text']}' @ {orig['start']}-{orig['end']} ({orig['label']})")
            if rec["suggestion"]:
                s = rec["suggestion"]
                print(f"    • Vorschlag: '{s['text']}' @ {s['start']}-{s['end']}")
            else:
                print("    • Kein Vorschlag per Token-Expand möglich")
            print()

    print(f"✅ Bereinigte Datei: {cleaned_path.resolve()}")
    print(f"✅ Fail-Liste:         {failed_path.resolve()}")

# Ausführen
process_file(input_path, cleaned_path, failed_path)


✅ Bereinigt: 14360 Einträge
⚠️ Fehlgeschlagen: 1260 Label-Zuordnungen

Beispiele der fehlgeschlagenen Labels mit Vorschlag:
 - Datei '801':
    • Original: '407286623469' @ 136-148 (VERTRAGSNUMMER)
    • Vorschlag: '407286623469Im' @ 136-150

 - Datei '802':
    • Original: '407588869318' @ 136-148 (VERTRAGSNUMMER)
    • Vorschlag: '407588869318Im' @ 136-150

 - Datei '803':
    • Original: '405822881747' @ 136-148 (VERTRAGSNUMMER)
    • Vorschlag: '405822881747Im' @ 136-150

 - Datei '804':
    • Original: '402140706469' @ 136-148 (VERTRAGSNUMMER)
    • Vorschlag: '402140706469Im' @ 136-150

 - Datei '805':
    • Original: '401404848848' @ 136-148 (VERTRAGSNUMMER)
    • Vorschlag: '401404848848Im' @ 136-150

✅ Bereinigte Datei: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/data/synthetic/synthetic_mails_option_a_cleaned.json
✅ Fail-Liste:         /Users/timonmartens/Library/CloudStorage/OneDrive-Persönli